In [1]:
from datetime import datetime
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [2]:
#1. Data
df = pd.read_csv("price_GOOG.CSV")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700
4,2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232100
...,...,...,...,...,...,...,...
3926,2020-03-25,1126.469971,1148.900024,1086.010010,1102.489990,1102.489990,4081500
3927,2020-03-26,1111.800049,1169.969971,1093.530029,1161.750000,1161.750000,3571700
3928,2020-03-27,1125.670044,1150.670044,1105.910034,1110.709961,1110.709961,3208500
3929,2020-03-30,1125.040039,1151.630005,1096.479980,1146.819946,1146.819946,2574100


In [3]:
# plt.plot(df["Date"].values, df["Adj Close"].values)  #pandas - loading data
# plt.show()

In [4]:
training_set = df.iloc[:, 5:6].values
training_set


array([[  49.982655],
       [  53.95277 ],
       [  54.495735],
       ...,
       [1110.709961],
       [1146.819946],
       [1162.810059]])

In [5]:
print(training_set.ndim)
print(training_set.shape)

2
(3931, 1)


In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled)

[[1.11307572e-04]
 [2.79949983e-03]
 [3.16714518e-03]
 ...
 [7.18337085e-01]
 [7.42787405e-01]
 [7.53614421e-01]]


In [7]:
X_train = [] #past 60'd price
Y_train = [] #today's price

for i in range(60, training_set_scaled.shape[0]):
    X_train.append(training_set_scaled[i-60:i, 0])
    Y_train.append(training_set_scaled[i,0])
    
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(X_train)
print(X_train.shape)
print(X_train.ndim)
print(Y_train)
print(Y_train.shape)

[[1.11307572e-04 2.79949983e-03 3.16714518e-03 ... 2.31683982e-02
  2.28850736e-02 2.79983798e-02]
 [2.79949983e-03 3.16714518e-03 1.63922502e-03 ... 2.28850736e-02
  2.79983798e-02 2.76543452e-02]
 [3.16714518e-03 1.63922502e-03 2.02036376e-03 ... 2.79983798e-02
  2.76543452e-02 2.86223649e-02]
 ...
 [8.71573188e-01 8.92123365e-01 8.87580004e-01 ... 7.34418374e-01
  7.12771286e-01 7.52896648e-01]
 [8.92123365e-01 8.87580004e-01 9.10296891e-01 ... 7.12771286e-01
  7.52896648e-01 7.18337085e-01]
 [8.87580004e-01 9.10296891e-01 9.09707812e-01 ... 7.52896648e-01
  7.18337085e-01 7.42787405e-01]]
(3871, 60)
2
[0.02765435 0.02862236 0.02446359 ... 0.71833709 0.74278741 0.75361442]
(3871,)


In [12]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] ,1))
print(X_train)
print(X_train.ndim)
print(X_train.shape)

[[[1.11307572e-04]
  [2.79949983e-03]
  [3.16714518e-03]
  ...
  [2.31683982e-02]
  [2.28850736e-02]
  [2.79983798e-02]]

 [[2.79949983e-03]
  [3.16714518e-03]
  [1.63922502e-03]
  ...
  [2.28850736e-02]
  [2.79983798e-02]
  [2.76543452e-02]]

 [[3.16714518e-03]
  [1.63922502e-03]
  [2.02036376e-03]
  ...
  [2.79983798e-02]
  [2.76543452e-02]
  [2.86223649e-02]]

 ...

 [[8.71573188e-01]
  [8.92123365e-01]
  [8.87580004e-01]
  ...
  [7.34418374e-01]
  [7.12771286e-01]
  [7.52896648e-01]]

 [[8.92123365e-01]
  [8.87580004e-01]
  [9.10296891e-01]
  ...
  [7.12771286e-01]
  [7.52896648e-01]
  [7.18337085e-01]]

 [[8.87580004e-01]
  [9.10296891e-01]
  [9.09707812e-01]
  ...
  [7.52896648e-01]
  [7.18337085e-01]
  [7.42787405e-01]]]
3
(3871, 60, 1)


In [13]:
# 2.Model & Traning

In [14]:
model = keras.Sequential([
    keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(units=50, return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(units=50, return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(units=50),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=1)
])

model.compile(
    optimizer = 'adam',
    loss = 'mse'  
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 60, 50)            10400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)               

In [15]:
# model.fit(X_train, Y_train, epochs = 100)
model.fit(X_train, Y_train, epochs = 10)

Train on 3871 samples
Epoch 1/10
3871/3871 [==============================] - 19s 5ms/sample - loss: 0.0072
Epoch 2/10
3871/3871 [==============================] - 16s 4ms/sample - loss: 0.0022
Epoch 3/10
3871/3871 [==============================] - 16s 4ms/sample - loss: 0.0018
Epoch 4/10
3871/3871 [==============================] - 16s 4ms/sample - loss: 0.0017
Epoch 5/10
3871/3871 [==============================] - 16s 4ms/sample - loss: 0.0015
Epoch 6/10
3871/3871 [==============================] - 15s 4ms/sample - loss: 0.0015
Epoch 7/10
3871/3871 [==============================] - 15s 4ms/sample - loss: 0.0014
Epoch 8/10
3871/3871 [==============================] - 14s 4ms/sample - loss: 0.0013
Epoch 9/10
3871/3871 [==============================] - 14s 4ms/sample - loss: 0.0014
Epoch 10/10
3871/3871 [==============================] - 14s 4ms/sample - loss: 0.0012


In [ ]:
#3.preict Data
print(df.head())
real_stock_data = df.iloc[-60:, 5:6].value
print(real_stock_data)

In [ ]:
X_test = []
for i in range(0, 60):
    val = df.iloc[-60-60+i:-60+i, 5:6].values
    val = sc.transform(val)
    X_test.append(val)
    
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

print("X_test.shape")
print(X_test.shape)
print(X_test[:1])

In [ ]:
predicted = model.predict(X_test)
predicted = sc.inverse_transform(predicted)

In [ ]:
plt.plot(real_stock_data, color='red', label = 'real_stock_data')
plt.plot(predicted, color="blue", label = "Predicted Stock price")
plt.xlabel("date")
